# Перенос стиля. Подготовка данных 3.

В этом файле формируем выборки предложений для обучения нейросети, а также обучаем бинарный классификатор fasttext, распознающий стиль предложения.

In [1]:
import nltk.data
import random
from nltk.tokenize import word_tokenize

In [2]:
DATA_PATH = './data/'

In [8]:
def load_sentences(filename):

    sentences = []
    with open(filename, encoding='UTF-8') as f:
        for line in f:
            sentences.append(line.strip('\n'))
    f.close()
    return sentences

In [4]:
tokenizer = nltk.data.load('tokenizers/punkt/russian.pickle')

In [9]:
sentences_push = load_sentences(DATA_PATH+"pushkin_rev.txt")

In [13]:
len(sentences_push)

5687

In [11]:
sentences_push = list(filter(lambda a: a != '.' and a !='??' and a !='?' and a !='!' and a!='', sentences_push))

In [12]:
sentences_cvet = load_sentences(DATA_PATH+"cvetaeva_rev.txt")
sentences_cvet = list(filter(lambda a: a != '.' and a !='??' and a !='?' and a !='!' and a!='', sentences_cvet))

In [14]:
len(sentences_cvet)

4940

In [20]:
import copy
import random

def save_sentences(list_sent, file_path):
    with open(file_path, 'w', encoding='UTF-8') as f:
        for s in list_sent:
            f.write(s + '\n')
    f.close() 

### Разбиение предложения на train, test и dev

In [22]:
datapath= 'data/poets/'
def split_dataset(sentences, temp_name, tr, d):
    sent_temp = copy.deepcopy(sentences)
    random.shuffle(sent_temp)
    train = sent_temp[:int(tr*(len(sent_temp)))]
    dev = sent_temp[int(tr*(len(sent_temp))):int(tr*(len(sent_temp)))+int(d*(len(sent_temp)))]
    test = sent_temp[int(tr*(len(sent_temp)))+int(d*(len(sent_temp))):]
    
    save_sentences(train, datapath+temp_name+'.train')
    save_sentences(dev, datapath+temp_name+'.dev')
    save_sentences(test, datapath+temp_name+'.test')

In [23]:
split_dataset(sentences_cvet, 'cvet2', 0.7, 0.2)

In [24]:
split_dataset(sentences_push, 'push2', 0.7, 0.2)

## FastText

Обучение классификатора

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
X = []
y = []


for s in sentences_cvet:
    X.append(s)
    y.append('__label__cvetaeva')
for s in sentences_push:
    X.append(s)
    y.append('__label__pushkin')

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [28]:
with open('fasttext_train_new.txt', 'w', encoding='UTF-8') as f_train:
    for x, y in zip(X_train, y_train):
        line = y + ' ' + x + '\n'
        f_train.write(line)
f_train.close()

with open('fasttext_test_new.txt', 'w', encoding='UTF-8') as f_test:
    for x, y in zip(X_test, y_test):
        line = y + ' ' + x + '\n'
        f_test.write(line)
f_test.close()

In [36]:
def test_param(train_file, test_file, param_dict):
    max_prec = 0
    max_lr = 0
    max_epoch = 0
    max_wng = 0
    max_dim = 0
    best_model = ''
    for lrate in param_dict['lr']:
        for epoch_num in param_dict['epoch']:
            for wng in param_dict['wordNgrams']:
                for dim_n in param_dict['dim']:
                    model1 = fasttext.train_supervised(input=train_file,
                                                       lr=lrate,
                                                       epoch=epoch_num,
                                                       wordNgrams=wng,
                                                       bucket=200000,
                                                       dim=dim_n)
                    
                    _, prec, rec = model1.test(test_file)
                    print('lr={}, epoch={}, wordNgrams={}, dim={}, precision={}, recall={}'.format(lrate,
                                                                                                   epoch_num,
                                                                                                   wng,
                                                                                                   dim_n,
                                                                                                   prec,
                                                                                                   rec))
                    if max_prec < prec:
                        max_prec = prec
                        max_lr = lrate
                        max_epoch = epoch_num
                        max_wng = wng
                        max_dim = dim_n
                        best_model = model1
    print('Best parameters: lr={}, epoch={}, wordNgrams={}, dim={}'.format(max_lr,
                                                                           max_epoch,
                                                                           max_wng,
                                                                           max_dim))
    return best_model

In [37]:
best_model = test_param('fasttext_train_new.txt', 'fasttext_test_new.txt', param_dict)

lr=0.01, epoch=25, wordNgrams=1, dim=10, precision=0.852304797742239, recall=0.852304797742239
lr=0.01, epoch=25, wordNgrams=1, dim=50, precision=0.8466603951081844, recall=0.8466603951081844
lr=0.01, epoch=25, wordNgrams=1, dim=100, precision=0.8466603951081844, recall=0.8466603951081844
lr=0.01, epoch=25, wordNgrams=1, dim=256, precision=0.845719661335842, recall=0.845719661335842
lr=0.01, epoch=25, wordNgrams=2, dim=10, precision=0.8231420507996237, recall=0.8231420507996237
lr=0.01, epoch=25, wordNgrams=2, dim=50, precision=0.819379115710254, recall=0.819379115710254
lr=0.01, epoch=25, wordNgrams=2, dim=100, precision=0.819379115710254, recall=0.819379115710254
lr=0.01, epoch=25, wordNgrams=2, dim=256, precision=0.8184383819379115, recall=0.8184383819379115
lr=0.01, epoch=25, wordNgrams=3, dim=10, precision=0.8099717779868297, recall=0.8099717779868297
lr=0.01, epoch=25, wordNgrams=3, dim=50, precision=0.8033866415804327, recall=0.8033866415804327
lr=0.01, epoch=25, wordNgrams=3, d

lr=0.1, epoch=50, wordNgrams=1, dim=50, precision=0.9078080903104422, recall=0.9078080903104422
lr=0.1, epoch=50, wordNgrams=1, dim=100, precision=0.9078080903104422, recall=0.9078080903104422
lr=0.1, epoch=50, wordNgrams=1, dim=256, precision=0.9078080903104422, recall=0.9078080903104422
lr=0.1, epoch=50, wordNgrams=2, dim=10, precision=0.9002822201317028, recall=0.9002822201317028
lr=0.1, epoch=50, wordNgrams=2, dim=50, precision=0.8974600188146754, recall=0.8974600188146754
lr=0.1, epoch=50, wordNgrams=2, dim=100, precision=0.8984007525870179, recall=0.8984007525870179
lr=0.1, epoch=50, wordNgrams=2, dim=256, precision=0.8974600188146754, recall=0.8974600188146754
lr=0.1, epoch=50, wordNgrams=3, dim=10, precision=0.8842897460018815, recall=0.8842897460018815
lr=0.1, epoch=50, wordNgrams=3, dim=50, precision=0.8842897460018815, recall=0.8842897460018815
lr=0.1, epoch=50, wordNgrams=3, dim=100, precision=0.8842897460018815, recall=0.8842897460018815
lr=0.1, epoch=50, wordNgrams=3, dim

In [38]:
#model = fasttext.train_supervised(input='fasttext_train_new.txt', lr=0.5, epoch=50, wordNgrams=1, bucket=200000,  dim=10)
best_model.test('fasttext_test_new.txt')

(1063, 0.9106302916274694, 0.9106302916274694)

In [39]:
best_model.save_model("pushkin_cvetaeva_new.bin")

## Обучение языковой модели KenLM

Обучали 3-грамную и 5-грамную языковые модели согласно https://github.com/kmario23/KenLM-training .

# Проверка кода 

In [29]:
import torch
import time
from data import load_dataset
from models import StyleTransformer, Discriminator
from train import train, auto_eval
import pickle

class Config():
    data_path = './data/poets/'
    log_dir = 'runs/exp'
    save_path = './save'
    pretrained_embed_path = './embedding/'
    device = torch.device('cuda' if True and torch.cuda.is_available() else 'cpu')
    discriminator_method = 'Multi' # 'Multi' or 'Cond'
    load_pretrained_embed = False
    min_freq = 1
    max_length = 90
    embed_size = 256
    d_model = 256
    h = 4
    num_styles = 2
    num_classes = num_styles + 1 if discriminator_method == 'Multi' else 2
    num_layers = 4
    batch_size = 20
    lr_F = 0.0001
    lr_D = 0.0001
    L2 = 0
    iter_D = 10
    iter_F = 5
    F_pretrain_iter = 500
    log_steps = 5
    eval_steps = 25
    learned_pos_embed = True
    dropout = 0
    drop_rate_config = [(1, 0)]
    temperature_config = [(1, 0)]

    slf_factor = 0.25
    cyc_factor = 0.5
    adv_factor = 1

    inp_shuffle_len = 0
    inp_unk_drop_fac = 0
    inp_rand_drop_fac = 0
    inp_drop_prob = 0
    
    acc_path = 'pushkin_cvetaeva_new.bin'
    ppl_path = 'poets_5_kenlm.binary'
    pos_label = '__label__cvetaeva'


def main():
    config = Config()
    train_iters, dev_iters, test_iters, vocab = load_dataset(config,
                                                            train_pos='cvet2.train', train_neg='push2.train',
                                                            dev_pos='cvet2.dev', dev_neg='push2.dev',
                                                            test_pos='cvet2.test', test_neg='push2.test')
    print('Vocab size:', len(vocab))
    model_F = StyleTransformer(config, vocab).to(config.device)
    model_D = Discriminator(config, vocab).to(config.device)
    print(config.discriminator_method)
    
    train(config, vocab, model_F, model_D, train_iters, dev_iters, test_iters)
    torch.save(model_F.state_dict(), 'modelF_trained')
    torch.save(model_D.state_dict(), 'modelD_trained')

In [30]:
import time
import numpy as np
import torchtext
from torchtext import data

from utils import tensor2text

class DatasetIterator(object):
    def __init__(self, pos_iter, neg_iter):
        self.pos_iter = pos_iter
        self.neg_iter = neg_iter

    def __iter__(self):
        for batch_pos, batch_neg in zip(iter(self.pos_iter), iter(self.neg_iter)):
            if batch_pos.text.size(0) == batch_neg.text.size(0):
                yield batch_pos.text, batch_neg.text

def load_dataset(config, train_pos='cvet2.train', train_neg='push2.train',
                 dev_pos='cvet2.dev', dev_neg='push2.dev',
                 test_pos='cvet2.test', test_neg='push2.test'):

    root = config.data_path
    TEXT = data.Field(batch_first=True, eos_token='<eos>')
    
    dataset_fn = lambda name: data.TabularDataset(
        path=root + name,
        format='tsv',
        fields=[('text', TEXT)]
    )

    train_pos_set, train_neg_set = map(dataset_fn, [train_pos, train_neg])
    dev_pos_set, dev_neg_set = map(dataset_fn, [dev_pos, dev_neg])
    test_pos_set, test_neg_set = map(dataset_fn, [test_pos, test_neg])

    TEXT.build_vocab(train_pos_set, train_neg_set, min_freq=config.min_freq)

    if config.load_pretrained_embed:
        start = time.time()
        
        vectors=torchtext.vocab.GloVe('6B', dim=config.embed_size, cache=config.pretrained_embed_path)
        TEXT.vocab.set_vectors(vectors.stoi, vectors.vectors, vectors.dim)
        print('vectors', TEXT.vocab.vectors.size())
        
        print('load embedding took {:.2f} s.'.format(time.time() - start))

    vocab = TEXT.vocab
        
    dataiter_fn = lambda dataset, train: data.BucketIterator(
        dataset=dataset,
        batch_size=config.batch_size,
        shuffle=train,
        repeat=train,
        sort_key=lambda x: len(x.text),
        sort_within_batch=False,
        device=config.device
    )

    train_pos_iter, train_neg_iter = map(lambda x: dataiter_fn(x, True), [train_pos_set, train_neg_set])
    dev_pos_iter, dev_neg_iter = map(lambda x: dataiter_fn(x, False), [dev_pos_set, dev_neg_set])
    test_pos_iter, test_neg_iter = map(lambda x: dataiter_fn(x, False), [test_pos_set, test_neg_set])

    train_iters = DatasetIterator(train_pos_iter, train_neg_iter)
    dev_iters = DatasetIterator(dev_pos_iter, dev_neg_iter)
    test_iters = DatasetIterator(test_pos_iter, test_neg_iter)
    
    return train_iters, dev_iters, test_iters, vocab

In [31]:
config = Config()

In [32]:
train_iters, dev_iters, test_iters, vocab = load_dataset(config,
                                                            train_pos='cvet2.train', train_neg='push2.train',
                                                            dev_pos='cvet2.dev', dev_neg='push2.dev',
                                                            test_pos='cvet2.test', test_neg='push2.test')

In [33]:
def inference(data_iter, raw_style):
    gold_text = []
    raw_output = []
    rev_output = []
    for batch in data_iter:
        inp_tokens = batch.text
        inp_lengths = get_lengths(inp_tokens, eos_idx)
        raw_styles = torch.full_like(inp_tokens[:, 0], raw_style)
        rev_styles = 1 - raw_styles



        gold_text += tensor2text(vocab, inp_tokens.cpu())


    return gold_text

In [34]:
def get_lengths(tokens, eos_idx):
    lengths = torch.cumsum(tokens == eos_idx, 1)
    lengths = (lengths == 0).long().sum(-1)
    lengths = lengths + 1 # +1 for <eos> token
    return lengths


In [35]:
test_iters

In [36]:
pad_idx = vocab.stoi['<pad>']
eos_idx = vocab.stoi['<eos>']

pos_iter = test_iters.pos_iter
neg_iter = test_iters.neg_iter

gold_text = zip(inference(neg_iter, 0), inference(pos_iter, 1))

In [37]:
inference(neg_iter, 0)

['но так. сей анджело сей злобный человек, сей грешник — был любим душою печальной и смиренной, душой, мучителем своим. он был давно <pad>',
 'сатирик и поэт наш и ты не тетушки моей. <pad>',
 'сказал и вышел вон, девицу оставя в к небесам ясный взор и чистую от палат спешит она в темницу. дверь отворилась ей; и брат ее глазам <pad>',
 'пускай я радости вкушаю не вполне; но ты, ты рождена для счастья. <pad>',
 'со вздохом витязь вкруг себя взирает грустными очами. о поле, кто тебя мертвыми чей борзый конь тебя в последний час кровавой кто на тебе со славой пал? <pad>',
 'над пастушка робко и фавна за собой все ближе, ближе слышит. уж она огонь его напрасны все старанья: ты но шумная волна красавицу река — ее могила… нет! лила <pad>',
 'восторгом витязь упоенный уже забыл людмилы пленной недавно милые томится сладостным взор его блестит, и, полный страстным он тает сердцем, он горит. но вот выходит он из <pad>',
 'счастлив, кто мил и страшен миру; о ком за песни, за дела гремит правдива

In [1]:
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import sentence_bleu

import fasttext
import pkg_resources
import kenlm
import math

In [3]:
acc_path='evaluator/pushkin_cvetaeva_new.bin'

In [7]:

classifier = fasttext.load_model(acc_path)

In [30]:
classifier.predict('олубей нити путь! мать. и младость отними сняла руками, тебя, волшебной луг, и «я и море! волшебной и принца луг, срок, твоей ночи! твоей ночи! ком целый щеку, твоей мной. нужно имена. и твоей волшебной имена. волшебной мать.')

(('__label__pushkin',), array([0.99694347]))